In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest


# Feature Engineering

In [2]:

df=pd.read_csv('tough_dataset.csv',encoding='latin1')
df.shape
pd.isnull(df['Hours/Shift']).sum()
for col_name in df.columns: #dropping the features having more than half missing values
 if (pd.isnull(df[col_name]).sum()>1475):
    df.drop([col_name],axis=1,inplace=True)


In [3]:
df.shape

(2946, 65)

In [4]:
sumna=pd.isnull(df).sum()
sumna.sort_values(ascending=False).tail(44)

LanguageDesireNextYear       667
CheckInCode                  627
VersionControl               614
IDE                          586
OperatingSystem              569
LanguageWorkedWith           499
HopeFiveYears                469
LastNewJob                   447
JobSearchStatus              435
YearsCodingProf              408
Preferred Skills             393
UndergradMajor               322
emp_length                   133
title                         64
FormalEducation               52
DevType                       45
Minimum Qual Requirements     20
Process Date                   4
Residency Requirement          4
Posting Updated                4
Posting Date                   4
YearsCoding                    3
To Apply                       1
Target A                       0
Target B                       0
LOAN_AMT                       0
Business Title                 0
Division/Work Unit             0
Job Description                0
Civil Service Title            0
last_credi

In [5]:
#random sampling the missing values
for feature in df.columns:

    df[feature+"random"]=df[feature]   
    df3=df[feature].dropna().sample(pd.isnull(df[feature]).sum(),random_state=0)
    df3.index=df[df[feature].isnull()].index
    df.loc[df[feature].isnull(),feature+"random"]=df3

In [6]:
#imputed dataset
df4=df.iloc[:,65:]

In [7]:
#One hot encoding the top 15 categories or less
for i in df4.iloc[:,4:].columns:
 if df4[i].value_counts().count()>100:
    df4.drop([i],axis=1,inplace=True)
df4.head(10)
df4.shape
for i in df4.iloc[:,4:].columns:
 if df4[i].value_counts().count()>15:
  top_15=[x for x in df4[i].value_counts().sort_values(ascending=False).head(15).index]
  for label in top_15:
   df4[label]=np.where(df4[i]==label,1,0)
 if df4[i].value_counts().count()<=15:
  top=[x for x in df4[i].value_counts().sort_values(ascending=False).index]
  for label in top:
   df4[label]=np.where(df4[i]==label,1,0)
        
 
df4.iloc[:,32:].head(50)
df5=df4.iloc[:,0:4]
df6=df4.iloc[:,36:]
finale=pd.concat([df5,df6],axis=1)
#one hot encoded data set
finale


 

,idrandom,Target Arandom,Target Brandom,LOAN_AMTrandom,"New York City residency is generally required within 90 days of appointment. However, City Employees in certain titles who have worked for the City for 2 continuous years may also be eligible to reside in Nassau, Suffolk, Putnam, Westchester, Rockland, or Orange County. To determine if the residency requirement applies to you, please discuss with the agency representative at the time of interview.",New York City Residency is not required for this position,NYCHA has no residency requirements.,New York City Residency is not required for this position.,This position is exempt from NYC residency requirements.,New York City residency is not required for this position.,...,17-Apr,17-Mar,16-Dec,18-Jul,18-Nov,18-Oct,18-Sep,18-Aug,17-Oct,18-May
0,68407277,42405.00,0,3600,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,68355089,60740.00,1,24700,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,68341763,51907.68,0,20000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66310712,51907.68,0,35000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68476807,35.00,1,10400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2941,68394526,67757.00,0,14000,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2942,68404684,67757.00,1,18675,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2943,68414567,54100.00,0,24000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2944,68373599,54100.00,0,2500,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Feature Selection

In [8]:
#selecting 25 best features for Category B
from sklearn.model_selection import train_test_split
XR=finale.iloc[:,3:]
yr=finale.iloc[:,2]
XR_train, XR_test, yr_train, yr_test = train_test_split(XR, yr, test_size=0.2)
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
ordered_rank_features=SelectKBest(score_func=chi2, k=100)
ordered_feature=ordered_rank_features.fit(XR_train,yr_train)
dfscores=pd.DataFrame(ordered_feature.scores_,columns=["Score"])
dfcolumns=pd.DataFrame(XR_train.columns)
features_rank=pd.concat([dfcolumns,dfscores],axis=1)
features_rank.columns=['Features','Score']
list=features_rank.nlargest(25,'Score')
list
XR_test_new=pd.DataFrame()
XR_train_new=pd.DataFrame()
for i in list["Features"]:
    XR_train_new[i]=XR_train[i] #new data with the important feature
for i in list["Features"]:
    XR_test_new[i]=XR_test[i]

XR_train.shape  
yr_train.shape
XR_train_new.shape


    

#from sklearn.ensemble import ExtraTreesClassifier
#model=ExtraTreesClassifier()
#model.fit(XR_train,yr_train)


(2356, 25)

In [9]:
#selecting 25 best features for Category A
from sklearn.model_selection import train_test_split
XRA=finale.iloc[:,3:]
yrA=finale.iloc[:,1]
XRA_train, XRA_test, yrA_train, yrA_test = train_test_split(XRA, yrA, test_size=0.2)
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest
ordered_rank_featuresA=SelectKBest(score_func=f_regression, k=100)
ordered_featureA=ordered_rank_featuresA.fit(XRA_train,yrA_train)
dfAscores=pd.DataFrame(ordered_featureA.scores_,columns=["Score"])
dfAcolumns=pd.DataFrame(XRA_train.columns)
features_rankA=pd.concat([dfAcolumns,dfAscores],axis=1)
features_rankA.columns=['Features','Score']
listA=features_rankA.nlargest(25,'Score')
listA
XRA_test_new=pd.DataFrame()
XRA_train_new=pd.DataFrame()
for i in listA["Features"]:
    XRA_train_new[i]=XRA_train[i] #new data with the important feature
for i in listA["Features"]:
    XRA_test_new[i]=XRA_test[i]
    


D:\cod4\New folder\envs\chatbot-workshop\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:302: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
D:\cod4\New folder\envs\chatbot-workshop\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:307: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
D:\cod4\New folder\envs\chatbot-workshop\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


# Creating Machine Learning Models

In [10]:
#Random Forest for target B
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
classifier=RandomForestClassifier()
mod=classifier.fit(XR_train_new,yr_train)
pred=classifier.predict(XR_test_new)
print(metrics.accuracy_score(yr_test,pred))
from sklearn.metrics import f1_score
f1_score(yr_test,pred,average='weighted')


0.7237288135593221


0.6886003941248587

In [11]:
#Decision Trees for target B
from sklearn import tree
from sklearn import metrics
dt=tree.DecisionTreeClassifier()
mod2=dt.fit(XR_train_new,yr_train)
pred2=dt.predict(XR_test_new)
print(metrics.accuracy_score(yr_test,pred2))
from sklearn.metrics import f1_score
f1_score(yr_test,pred2,average='weighted')

0.7237288135593221


0.6919508799427955

In [12]:
#XGboost Classifier for target B
import xgboost as xgb
from sklearn.metrics import f1_score
xgclass=xgb.XGBClassifier()
xgclass.fit(XR_train_new,yr_train)
predxgbB=xgclass.predict(XR_test_new)
print(metrics.accuracy_score(yr_test,predxgbB))
f1_score(yr_test,predxgbB,average='weighted')

[19:52:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

D:\cod4\New folder\envs\chatbot-workshop\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



0.764406779661017


D:\cod4\New folder\envs\chatbot-workshop\lib\site-packages\xgboost\data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


0.7137424604570409

In [13]:
#Decision Trees for Target A
from sklearn import tree
from sklearn import metrics
from math import sqrt
from sklearn.metrics import mean_squared_error
dtA=tree.DecisionTreeRegressor()
mod2A=dtA.fit(XRA_train_new,yrA_train)
pred2A=dt.predict(XRA_test_new)
rms2=sqrt(mean_squared_error(yrA_test,pred2A))
rms2



66965.29611957997

In [14]:
#Random Forest for target A
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
regressorA=RandomForestRegressor()
modA=regressorA.fit(XRA_train_new,yrA_train)
predA=regressorA.predict(XRA_test_new)

rms1=sqrt(mean_squared_error(yrA_test,predA))
rms1

#print(metrics.accuracy_score(yrA_test,predA))

27333.15751450374

In [15]:
#XGboost Regression for Target A
import xgboost as xgb
xgreg=xgb.XGBRegressor()
xgreg.fit(XRA_train_new,yrA_train)
predxgbA=xgreg.predict(XRA_test_new)
data_dmatrix = xgb.DMatrix(data=XRA_train_new,label=yrA_train)
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,'max_depth': 5, 'alpha': 10}
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
cv_results.head()
print((cv_results["test-rmse-mean"]).tail(1))



D:\cod4\New folder\envs\chatbot-workshop\lib\site-packages\xgboost\data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


[19:52:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
49    25362.580729
Name: test-rmse-mean, dtype: float64
